In [1]:
from pathlib import Path
import json
import os

from rdflib import URIRef, Namespace

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Template
from buildingmotif.semantic_graph_synthesizer.bindings_utils import unify_bindings, evaluate_bindings
from buildingmotif.semantic_graph_synthesizer.classes import TokenizedLabel
from buildingmotif.semantic_graph_synthesizer.semantic_graph_synthesizer import SemanticGraphSynthesizer
from buildingmotif.semantic_graph_synthesizer.bipartite_token_mapper import BipartiteTokenMapper

import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

BLDG = Namespace("urn:building/")
PROJECT_DIR = Path(os.getcwd()).resolve().parent
bm = BuildingMOTIF("sqlite://")

In [2]:
# These are our Templates
Library.load(ontology_graph="https://github.com/BrickSchema/Brick/releases/download/nightly/Brick.ttl")
equiment_templates = Library.load(directory=str(PROJECT_DIR / "gabes_points" / "equiment_templates"))
templates = equiment_templates.get_templates()

for template in templates:
    template.inline_dependencies()
    print(f"{template.name} =======================")
    print(template.body.serialize(format="ttl"))
    print()

2023-10-09 12:46:53,734 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6922
2023-10-09 12:46:53,736 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6924
2023-10-09 12:46:53,737 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6926
2023-10-09 12:46:53,737 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6928
2023-10-09 12:46:53,738 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6930
2023-10-09 12:46:53,739 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2023-10-09 12:46:53,740 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6972
2023-10-09 12:46:53,741 | root |  WARNING: Warning: could not find dependee n4344d56466c2426fbb28184a3917d0f4b6974
2023-10-09 12:46:53,741 | root |  WARNING: Warning: could not find depe

SaTmpEntity =======================
@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a ns1:Fan_Coil_Unit ;
    ns1:hasPoint <urn:___param___#sat> .

<urn:___param___#sat> a ns1:Supply_Air_Temperature_Sensor .



fcu =======================
@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a ns1:Fan_Coil_Unit ;
    ns1:feeds <urn:___param___#zone> ;
    ns1:hasPart <urn:___param___#chw_coil>,
        <urn:___param___#hw_coil> ;
    ns1:hasPoint <urn:___param___#occ_clg_sp>,
        <urn:___param___#occ_cmd>,
        <urn:___param___#occ_htg_sp>,
        <urn:___param___#occ_status>,
        <urn:___param___#supply_temp>,
        <urn:___param___#unocc_clg_sp>,
        <urn:___param___#unocc_htg_sp> .

<urn:___param___#zone> a ns1:HVAC_Zone ;
    ns1:hasPart <urn:___param___#room> .



room =======================
@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a ns1:Space ;
    ns1:hasPoint <urn:__

In [3]:
# These are our points - there's alot
with open(PROJECT_DIR / "gabes_points" / "tokens.json") as f:
    labels_and_tokens = json.load(f)
    labels = [TokenizedLabel.from_dict(x) for x in labels_and_tokens]

len(labels)

5319

In [4]:
# This is what a point looks like
label = labels[0]

label

:BuildingName_02:FCU503_ChwVlvPos:
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - 503 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
    - :BuildingName_02:FCU503_ChwVlvPos (type https://brickschema.org/schema/Brick#Chilled_Water_Valve)

In [5]:
# We are going to use this to get bindings. 
sgs = SemanticGraphSynthesizer()

In [6]:
# here's how to get one labels bindings
bindings = sgs.find_bindings_for_label(templates, labels[0])

print(bindings.template.name)
print(bindings.bindings)
print(bindings.cost)

fcu
{'name': 503 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit), 'chw_coil-chw_vlv': :BuildingName_02:FCU503_ChwVlvPos (type https://brickschema.org/schema/Brick#Chilled_Water_Valve)}
Cost(edge_cost=0.0, params_dropped=16, tokens_dropped=2)


In [7]:
# we can group labels into labelsets to greatly reduce the ammount of work doing this in bulk takes. 
labelsets = sgs._group_labels_by_tokens(labels)

labelsets[0]

LabelSet(token_classes=[rdflib.term.URIRef('https://brickschema.org/schema/Brick#Air_Handling_Unit'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Chilled_Water_Valve'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Entity'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Fan_Coil_Unit')], labels=[:BuildingName_02:FCU503_ChwVlvPos:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - :BuildingName_02:FCU503_ChwVlvPos (type https://brickschema.org/schema/Brick#Chilled_Water_Valve)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 503 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
, :BuildingName_01:FCU362_ChwVlvPos:
    - 01 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - :BuildingName_01:FCU362_ChwVlvPos (type https://brickschema.org/schema/Brick#Chilled_Water_Valve)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 362 (type https://brickschema.or

In [8]:
# See? only 13 labelsets total
len(labelsets)

13

In [9]:
# find_bindings_for_labels is like find_bindings_for_label, but bulk. It uses labelsets to go fast. 
bindings_list = sgs.find_bindings_for_labels(templates, labels)

len(bindings_list)

5319

In [10]:
# we can build out more meaningful bindings by unifying the ones on the same template with the same name
unified_bindings_list = unify_bindings(bindings_list)

unified_bindings_list[0]

UnifiedBindings(labels=[:BuildingName_02:FCU503_ChwVlvPos:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - :BuildingName_02:FCU503_ChwVlvPos (type https://brickschema.org/schema/Brick#Chilled_Water_Valve)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 503 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
, :BuildingName_02:FCU503_EffOcc:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 503 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
    - :BuildingName_02:FCU503_EffOcc (type https://brickschema.org/schema/Brick#Occupancy_Status)
, :BuildingName_02:FCU503_UnoccHtgSpt:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 503 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
    - :BuildingName_02:FCU503_UnoccHtgS

In [11]:
# much fewer but more exciting!
len(unified_bindings_list)

544

In [12]:
# See? Very cool.
unified_bindings = unified_bindings_list[5]

unified_bindings

UnifiedBindings(labels=[:BuildingName_02:FCU530_ChwVlvPos:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - :BuildingName_02:FCU530_ChwVlvPos (type https://brickschema.org/schema/Brick#Chilled_Water_Valve)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 530 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
, :BuildingName_02:FCU530_EffOcc:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 530 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
    - :BuildingName_02:FCU530_EffOcc (type https://brickschema.org/schema/Brick#Occupancy_Status)
, :BuildingName_02:FCU530_UnoccHtgSpt:
    - 02 (type https://brickschema.org/schema/Brick#Air_Handling_Unit)
    - BuildingName (type https://brickschema.org/schema/Brick#Entity)
    - 530 (type https://brickschema.org/schema/Brick#Fan_Coil_Unit)
    - :BuildingName_02:FCU530_UnoccHtgS

In [13]:
# each unified binding can be evaluated into a graph. 
model = evaluate_bindings(unified_bindings)
if isinstance(model, Template):
    _, graph = model.fill(BLDG)
else:
    graph = model

print(graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/530> a brick:Fan_Coil_Unit ;
    brick:feeds <urn:building/zone_c413df9c> ;
    brick:hasPart <urn:building/chw_coil_640b4e5c>,
        <urn:building/hw_coil_5f91f070> ;
    brick:hasPoint <urn:building/:BuildingName_02:FCU530_EffOcc>,
        <urn:building/:BuildingName_02:FCU530_OccClgSpt>,
        <urn:building/:BuildingName_02:FCU530_OccCmd>,
        <urn:building/:BuildingName_02:FCU530_OccHtgSpt>,
        <urn:building/:BuildingName_02:FCU530_UnoccClgSpt>,
        <urn:building/:BuildingName_02:FCU530_UnoccHtgSpt>,
        <urn:building/supply_temp_0abdd5f3> .

<urn:building/:BuildingName_02:FCU530_ChwVlvPos> a brick:Chilled_Water_Valve ;
    brick:hasPoint <urn:building/chw_coil-position_d9d2e728> .

<urn:building/:BuildingName_02:FCU530_EffOcc> a brick:Occupancy_Status .

<urn:building/:BuildingName_02:FCU530_HwVlvPos> a brick:Hot_Water_Valve ;
    brick:hasPoint <urn:building/hw_coil-position_cf362cf3> .



/Users/heslinge/Repos/BuildingMotif/buildingmotif/dataclasses/template.py:372: UserWarning: Parameters "hw_coil, room-relative_humidity, zone, supply_temp, room, hw_coil-position, chw_coil-position, chw_coil" were not provided during evaluation
  warnings.warn(
